In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import pickle
from joblib import Parallel, delayed
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import roc_auc_score, precision_score
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

plt.style.use('seaborn')
sns.set(font_scale=2.5)


#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# Fixed to static seed 42.
import os
import random
import numpy as np
import tensorflow.compat.v1 as tf
import logging


SEED =42

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed) # ver1
    # tf.random.set_seed(seed) # ver 2
    np.random.seed(seed)
    
def set_global_determinism(seed=SEED, fast_n_close=False):
    """
        Enable 100% reproducibility on operations related to tensor and randomness.
        Parameters:
        seed (int): seed value for global randomness
        fast_n_close (bool): whether to achieve efficient at the cost of determinism/reproducibility
    """
    set_seeds(seed=seed)
    if fast_n_close:
        return

    logging.warning("*******************************************************************************")
    logging.warning("*** set_global_determinism is called,setting full determinism, will be slow ***")
    logging.warning("*******************************************************************************")

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    # https://www.tensorflow.org/api_docs/python/tf/config/threading/set_inter_op_parallelism_threads
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

    
# set_seeds(42)
# set_global_determinism(42)

set_seeds(42)
set_global_determinism(42)

# Data Load

In [3]:
# Preoperative data
data = pd.read_csv('data/train.csv')
data.drop(['Unnamed: 3','Unnamed: 7','Patient Initials', 'Histologic type'], axis=1, inplace=True)
data.head()

,Age,Menopause,Method,Grade,CA125 (IU/ml),Myometrial invasion depth,Tumor size (largest diameter)
0,54,Yes,Dilatation/currettage,I,14.94,Less than 50% (< 1/2),3
1,55,Yes,Dilatation/currettage,I,20.38,None,1.7
2,45,No,Dilatation/currettage,I,43.47,Less than 50% (< 1/2),1.25
3,62,Yes,Dilatation/currettage,I,13.83,Less than 50% (< 1/2),4.7
4,55,Yes,Dilatation/currettage,I,9.02,Less than 50% (< 1/2),1.3


In [4]:
# PostOperative data
label_data = pd.read_csv('data/label.csv')
label_data.drop(['Unnamed: 7'], axis=1, inplace=True)
label_data.head()

,Route,Stage,Histologic diagnosis,Grade,Myometrial invasion depth,Tumor size (largest diameter),Extrauterine involvement,Lymphovascular space invasion (LVSI),Metastasis of pelvic lymph node,Metastasis of para-aortic lymph node,LN metastasis
0,Laparoscopic,Ia,Endometrioid,I,Less than 50%,2.5,none,No,0,0,no
1,Laparoscopic,Ia,Endometrioid,I,Less than 50%,1.0,none,No,0,0,no
2,Laparoscopic,Ia,Endometrioid,I,Less than 50%,3.8,none,No,0,0,no
3,Laparoscopic,Ia,Endometrioid,I,Less than 50%,4.0,none,No,0,0,no
4,Laparoscopic,Ia,Endometrioid,I,Less than 50%,2.0,none,No,0,0,no


# Data preprocessing

### Add 'pre_GN' and 'post_GN' variabels

In [5]:
def pre_GN(grade, invasion):
    if grade == 'I' and invasion == 'None': # Group 1
        return 0
    elif grade == 'II' and invasion == 'None': # Group 2
        return 1
    elif grade == 'I' and invasion == 'Less than 50% (< 1/2)': # Group 3
        return 2
    elif grade == 'II' and invasion == 'Less than 50% (< 1/2)': # Group 4
        return 3
    else: # Others
        return 4
    
def post_GN(stage, grade, invasion):
    if stage == 'Ia':
        if grade == 'I' and invasion == 'None': # Group 1
            return 0
        elif grade == 'II' and invasion == 'None': # Group 2
            return 1
        elif grade == 'I' and invasion == 'Less than 50%': # Group 3
            return 2
        elif grade == 'II' and invasion == 'Less than 50%': # Group 4
            return 3
        else: # others. Stage is 1A but Grade is III or invasion is More than 50%.
            return 4
    else: # others. All if Stage is not 1A.
        return 4


data['pre_GN'] = data.apply(lambda x: pre_GN(x['Grade'], x['Myometrial invasion depth']), axis=1)
data['post_GN'] = label_data.apply(lambda x: post_GN(x['Stage'], x['Grade'], x['Myometrial invasion depth']), axis=1)

### Processing the 'Tumor size' .value(null)
Fill in with the average tumor size for each post-operative invasion depth group
* None: 1.41
* Less than 50%: 2.44
* More than 50%: 2.95

In [6]:
data['Tumor size (largest diameter)'].unique()

array(['3', '1.7', '1.25', '4.7', '1.3', '2.3', '5.5', '0.6', '2', '.',
       '3.7', '4.2', '1.5', '2.9', '2.2', '3.3', '5', '2.8', '0', '2.5',
       '1.8', '5.8', '0.1', '4', '1', '2.6', '7.58', '3.4', '2.1', '3.5',
       '1.2', '1.1', '4.5', '3.6', '0.9', '5.7', '3.1', '2.7', '0.8',
       '2.4', '4.1', '1.6', '1.9', '8.3', '3.2', '4.4', '1.45', '17',
       '3.9', '0.5'], dtype=object)

In [7]:
label_data['Tumor size (largest diameter)'].describe()

count    252.000000
mean       2.098413
std        1.592011
min        0.000000
25%        0.800000
50%        2.000000
75%        3.000000
max        8.200000
Name: Tumor size (largest diameter), dtype: float64

In [8]:
None_num = round(label_data[label_data['Myometrial invasion depth']=='None']['Tumor size (largest diameter)'].mean(), 2)
Less_num = round(label_data[label_data['Myometrial invasion depth']=='Less than 50%']['Tumor size (largest diameter)'].mean(), 2)
More_num = round(label_data[label_data['Myometrial invasion depth']=='More than 50%']['Tumor size (largest diameter)'].mean(), 2)

print(f"None_num: {None_num}, Less_num: {Less_num}, More_num: {More_num}")

None_num: 1.41, Less_num: 2.45, More_num: 2.95


In [9]:
for i in range(len(data)):
    if data['Tumor size (largest diameter)'][i] == '.': # If Tumor size is '.',
        # If the invasion depth is None, replace with the average value of 1.41
        if data['Myometrial invasion depth'][i] == 'None':
            data['Tumor size (largest diameter)'][i] = None_num
        # If the invasion depth is less than 50%, replace with the average value of 2.44
        elif data['Myometrial invasion depth'][i] == 'Less than 50% (< 1/2)':
            data['Tumor size (largest diameter)'][i] =  Less_num

In [10]:
data['Tumor size (largest diameter)'].unique()

array(['3', '1.7', '1.25', '4.7', '1.3', '2.3', '5.5', '0.6', '2', 2.45,
       '3.7', '4.2', '1.5', '2.9', '2.2', 1.41, '3.3', '5', '2.8', '0',
       '2.5', '1.8', '5.8', '0.1', '4', '1', '2.6', '7.58', '3.4', '2.1',
       '3.5', '1.2', '1.1', '4.5', '3.6', '0.9', '5.7', '3.1', '2.7',
       '0.8', '2.4', '4.1', '1.6', '1.9', '8.3', '3.2', '4.4', '1.45',
       '17', '3.9', '0.5'], dtype=object)

In [11]:
# Convert dtype of tumor size to float
data['Tumor size (largest diameter)'] = data['Tumor size (largest diameter)'].astype('float64')

In [18]:
data['Myometrial invasion depth'] = data['Myometrial invasion depth'].replace('Less than 50% (< 1/2)', 'Less than 50%')

label_data['Myometrial invasion depth2'] = label_data['Myometrial invasion depth']
label_data['Myometrial invasion depth2'] = label_data['Myometrial invasion depth2'].replace('None', 0)
label_data['Myometrial invasion depth2'] = label_data['Myometrial invasion depth2'].replace('Less than 50%', 1)
label_data['Myometrial invasion depth2'] = label_data['Myometrial invasion depth2'].replace('More than 50%', 2)

label_data['Grade2'] = label_data['Grade']
label_data['Grade2'] = label_data['Grade2'].replace('I', 0)
label_data['Grade2'] = label_data['Grade2'].replace('II', 1)
label_data['Grade2'] = label_data['Grade2'].replace('others', 2)

# Utils

In [20]:
# Function to separate data into df_P and df_F, and decide the number of splits
def make_seperate_n_splits(daf):
    # Separate into df_P where 'post_GN' is True
    df_P = daf[daf['post_GN'] == True]
    df_P.reset_index(drop=True, inplace=True)

    # Separate into df_F where 'post_GN' is False
    df_F = daf[daf['post_GN'] == False]
    df_F.reset_index(drop=True, inplace=True)
    
    # Determine the number of splits based on the length of each category
    daf_len_True = len(daf[daf['post_GN'] == True])
    daf_len_False = len(daf[daf['post_GN'] == False])
    n_split = round(daf_len_False / daf_len_True, 0) # Calculate n_split, rounding to ensure it's an integer
    n_split = int(n_split)
    
    return df_P, df_F, n_split

# Function to create balanced datasets ensuring equal representation from minority class (e.g., False: 144, True: 57)
def make_balanced_dataset(n_split, df_F):
    data_kf = KFold(n_splits=n_split, shuffle=True, random_state=50)
    df_list = []
    # Create datasets with balanced classes using K-Fold cross-validation on the majority class
    for train_index, test_index in data_kf.split(df_F):
        X_train, X_test = df_F.iloc[train_index, :], df_F.iloc[test_index, :]

        # Combine the test split of majority class with the entire minority class, then reset index
        daf = pd.concat([X_test, df_P], axis=0)
        daf.reset_index(drop=True, inplace=True)

        df_list.append(daf)
        
    return df_list


# Sort dictionary values in descending order and select models based on base score
def make_max_score_model_selection(lists, base_score):
    # Create a boolean array where True if the score is greater than or equal to base_score
    k = np.array(lists) >= base_score
    
    # Collect indices of True values
    model_index = []
    for i in range(len(k)):
        if k[i] == True:
            model_index.append(i)
            
    return model_index

# Return True if the value is greater than 0.5, otherwise False
def make_valuessss(values):
    if values > 0.5:
        return True
    else:
        return False
    
# Calculate and return the average of predict_proba from selected models
def cal_pred_proba_test(model_index):
    # Select models that meet the condition based on model_index
    selected_model = []
    for i in model_index:
        selected_model.append(LR_random_list[i])
        
    # Calculate the average predict_proba from selected models
    pred_proba = []
    # Store each model's predict_proba result into a list
    for j in range(len(selected_model)):
        pred_proba.append(selected_model[j].predict_proba(X_test))
        
    # Compute the mean of the stored predict_proba results
    c = 0
    for j in range(len(pred_proba)):
        c += pred_proba[j]
    mean_score = c / len(pred_proba)  # Average out the accumulated probabilities
    
    # Convert average scores to DataFrame and calculate mean y_pred
    mean_score = pd.DataFrame(mean_score)    
    cal_mean_score_list.append(mean_score[1])
    
    # Apply threshold to determine the final prediction labels
    y_pred = mean_score[1].apply(make_valuessss)
    
    return y_pred





def make_cm_list_5(cm_list):    
    TP_list, FN_list, FP_list, TN_list = [], [], [], []
    for i in range(5):    
        TP, FN, FP, TN = cm_list[i].ravel()
        TP_list.append(TP)
        FN_list.append(FN)
        FP_list.append(FP)
        TN_list.append(TN)

    sum_TP = sum(TP_list)
    sum_FN = sum(FN_list)
    sum_FP = sum(FP_list)
    sum_TN = sum(TN_list)

    print(f'TP: {sum_TP},  FP: {sum_FP}, FN: {sum_FN}, TN: {sum_TN}')
    
    y_true = []
    # 0: others
    for i in range(sum_FP+sum_TN): # others total count (False total count FP + TN)
        y_true.append(0)

    for i in range(sum_TP+sum_FN): # Group 1 Total Count (True Total Count TP + FN)
        y_true.append(1)

    y_pred = []
    for i in range(sum_TN): # TN
        y_pred.append(0)

    for i in range(sum_FP): # FP
        y_pred.append(1)

    for i in range(sum_FN): # FN
        y_pred.append(0)

    for i in range(sum_TP): # TP
        y_pred.append(1)
    
    roc_auc_scores = round(roc_auc_score(y_true, y_pred), 4)
    
    print(round(roc_auc_score(y_true, y_pred), 4))
    
    return sum_TP, sum_FP, sum_FN, sum_TN, roc_auc_scores




def make_imputation_mice_lr_invasion(X_train, X_test_data):
    X_train_index = X_train.index    
    X_train_columns = X_train.columns
    
    X_test_data_idx = X_test_data.index
    X_test_data_col = X_test_data.columns
    
    lr = LinearRegression()
    imputer = IterativeImputer(estimator=lr, missing_values=np.nan, max_iter=10, verbose=0, imputation_order='random',random_state=42)
    
    # Handle train
    X_train = imputer.fit_transform(X_train)    
    X_train = pd.DataFrame(X_train, index = X_train_index, columns = X_train_columns)
        
    
    
    # Predicted invasion depth in X_test_data.
    X_test_data = pd.DataFrame(X_test_data, columns=X_train.columns)
    X_test_data.loc[:, 'Myometrial invasion depth'] = np.nan # Handling test set-wide missing
    
    X_test_data = imputer.transform(X_test_data)
    X_test_data = pd.DataFrame(X_test_data, index = X_test_data_idx, columns = X_test_data_col)

    
    return X_train, X_test_data


def calc_valid_auc(sum_TP, sum_FP, sum_FN, sum_TN):
    y_true = []
    # 0: others
    for i in range(sum_FP+sum_TN): # others total count (False total count FP + TN)
        y_true.append(0)

    for i in range(sum_TP+sum_FN): # Group 1 Total Count (True Total Count TP + FN)
        y_true.append(1)

    y_pred = []
    for i in range(sum_TN): # TN
        y_pred.append(0)

    for i in range(sum_FP): # FP
        y_pred.append(1)

    for i in range(sum_FN): # FN
        y_pred.append(0)

    for i in range(sum_TP): # TP
        y_pred.append(1)
    
    try:
        roc_auc_scores = round(roc_auc_score(y_true, y_pred), 4)
    except:
        roc_auc_scores=0.5    
    
    return roc_auc_scores

In [21]:
def make_logistic(X_data, y_label, y_label_len, i, q):
    model_kf = KFold(n_splits = y_label_len, shuffle = True, random_state = 50)  

     # Train a Logistic Regression model on each balanced dataset
    for idx, (train_index, test_index) in enumerate(model_kf.split(X_data)):
        X_train_model, X_test_model = X_data[train_index], X_data[test_index]
        y_train_model, y_test_model = y_label[train_index], y_label[test_index]

        LR_clf = LogisticRegression()

        solvers = ['newton-cg', 'lbfgs', 'liblinear']
        penalty = ['l2', 'l1']
        c_values = [1000, 100, 10, 1.0, 0.1, 0.01, 0.001] # Range of C values

        # define grid search
        grid = dict(solver=solvers,penalty=penalty,C=c_values)
        
        
        # Create a RandomizedSearchCV object to tune the Logistic Regression
        LR_random = RandomizedSearchCV(estimator=LR_clf,
                                       param_distributions=grid,
                                       n_iter=40,  # Number of parameter settings sampled
                                       scoring='f1_macro',  # Scoring metric
                                       n_jobs=-1,  # Use all cores
                                       cv=3,  # 3-fold cross-validation
                                       random_state=42,
                                       refit=True,
                                       return_train_score=True)
                

        grid_result = LR_random.fit(X_train_model, y_train_model)

        # predict
        pred_train_model = grid_result.best_estimator_.predict(X_train_model)
        y_pred_model = grid_result.best_estimator_.predict(X_test_model)
        
        X_valid_list.append(X_test_model) # valid set
        y_valid_list.append(y_test_model)
        
        LR_random_list.append(grid_result.best_estimator_)


        train_accuracy.append(accuracy_score(y_train_model, pred_train_model))
        test_accuracy.append(accuracy_score(y_test_model, y_pred_model))
        macro_average.append(f1_score(y_test_model, y_pred_model, average='macro'))
        True_recall_list.append(recall_score(y_test_model, y_pred_model))
        # roc_auc_list.append(roc_auc_score(y_test_model, y_pred_model))
        precision_list.append(precision_score(y_test_model, y_pred_model))

        y_test_list.append(np.array(y_test_model))
        y_pred_list.append(y_pred_model)

# Data processing function

In [22]:
from sklearn.preprocessing import StandardScaler

# n = 0: group 1,n = 1: group 2
def data_processing(data, n):
    numerical_features = data.select_dtypes(include=['int64', 'float64']).iloc[:, :-2]

    scaler = StandardScaler()
    scaler.fit(numerical_features)
    numerical_features_scaled = scaler.transform(numerical_features)

    numerical_features_scaled = pd.DataFrame(numerical_features_scaled, columns=numerical_features.columns)
    numerical_features_scaled = pd.concat([numerical_features_scaled, data.iloc[:, -2:]], axis=1)

    # One-Hot encoding the categorical features using get_dummies()
    # select categorical features
    categorical_features = data.select_dtypes(include=['object'])

    # Grade I: 0, Grade II: 1
    categorical_features['Grade'] = categorical_features['Grade'].replace('I', 0)
    categorical_features['Grade'] = categorical_features['Grade'].replace('II', 1)
    categorical_features['Myometrial invasion depth'] = categorical_features['Myometrial invasion depth'].replace('None', 0)
    categorical_features['Myometrial invasion depth'] = categorical_features['Myometrial invasion depth'].replace('Less than 50%', 1)
    categorical_features = pd.get_dummies(categorical_features)

    combined = pd.concat([numerical_features_scaled, categorical_features], axis=1)

    # seperate features and target
    X = combined
        
    # To use the Invasion depth value as is.
    X['pre_Myometrial invasion depth'] = X['Myometrial invasion depth']        
    # X['pre_Grade'] = X['Grade'] 
 
    
    y = (X['post_GN'] == n)
    print(y.value_counts())
    
    
    X.drop(columns=['pre_GN', 'post_GN'], inplace=True)


    return X, y

# NPM1

In [23]:
from sklearn.model_selection import StratifiedKFold
import shap

seed = 955

best_roc_auc_scores = 0
best_sum_TP =0
best_sum_FP =0
best_sum_FN =0
best_sum_TN =0
best_z = 0
roc_auc_scores_valid_list = []
base_auc_score_list=[]
score_auc_count_list=[]
roc_auc_scores_test_list_list=[]
base_score_list_list=[]
max_count_score_list_list=[]

base_score_rs_list = [] # Average of Score(Max((TP+TN)-(FP+FN))*0.999 for all folds by random state

valid_score_list = []
test_score_list = []

test_index_list = []
cal_mean_score_list = []


roc_auc_scores_test_list = []
base_score_list = []
max_count_score_list = []
for q in range(4): # q is the Group that predicts
    X, y = data_processing(data, q)  
    

    
    print(f'************ Group {q+1} ***************')

    # Number of splits, shuffle or not, and seed settings
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)    

    cm_valid_list = []
    cm_test_list = []
    None_concordance_all = 0
    Less_concordance_all = 0
    
    columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    TP_result = pd.DataFrame(columns = columns)
    FN_result = pd.DataFrame(columns = columns)
    FP_result = pd.DataFrame(columns = columns)
    TN_result = pd.DataFrame(columns = columns)
    result_cm_list=[]
    original_test_inv_data_list = []
    label_y_test_list = []
    X_test_data_pre_invasion = []
    prob_test_inv_data_list = []

    # Split the train and test datasets by the number of split steps each time.
    for n, (train_index, test_index) in enumerate(skf.split(X, y)):    
        train_accuracy = []
        test_accuracy = []
        macro_average = []
        y_test_list = []
        y_pred_list = []
        True_recall_list = []
        roc_auc_list = []
        precision_list = []
        best_estimator = []
        max_count_list = []
        valid_roc_auc_scores_list=[]
        
        X_train, X_test_data = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]        
        # Adding label invasion depth
        label_y_train, label_y_test = label_data.loc[train_index, 'Myometrial invasion depth2'], label_data.loc[test_index, 'Myometrial invasion depth2']

        if n==1:
            print(X_train.columns)

        test_index_list.append(test_index)

        # X_train의 index
        trn_idx = X_train.index
        tst_idx = X_test_data.index                        


        # handle train set missing (invasion depth)
        # Extract indexes with mismatched pre- and post-op invasions
        not_concordance_invasion_idx = X_train[(X_train.loc[trn_idx, 'Myometrial invasion depth'] != label_data.loc[trn_idx, 'Myometrial invasion depth2'])].index
        # Handling missing data with mismatched perioperative invasions
        X_train.loc[not_concordance_invasion_idx, 'Myometrial invasion depth'] = np.nan
        num_missing_invasion = X_train['Myometrial invasion depth'].isnull().sum()        


        # invasion imputation
        X_train, X_test_data = make_imputation_mice_lr_invasion(X_train, X_test_data)      

        # Compare matching on invasion depth imputaton
        # label invasion depth (original)
        original_test_inv_data = np.where(X_test_data['Myometrial invasion depth'] >= 0.5, 1, 0).tolist() # (이전 값) 0.5 이상은 1, 0.5이하는 0 
        prob_test_inv_data = X_test_data['Myometrial invasion depth']
        
        original_test_inv_data_list.extend(original_test_inv_data)
        label_y_test_list.extend(label_y_test)
        prob_test_inv_data_list.extend(prob_test_inv_data)


        X_test_data_pre_invasion.extend(X_test_data['pre_Myometrial invasion depth'])


        # Remove pre Myometrial invasion depth 
        X_train = X_train.drop(columns=['pre_Myometrial invasion depth'])
        X_test_data = X_test_data.drop(columns=['pre_Myometrial invasion depth'])


        if n==1:
            print(X_train.columns)


        X_test = np.array(X_test_data)
        
        # X_train, y_train by column, and then append 
        daf = pd.concat([X_train, y_train], axis=1)

        # Determine the number of df_P, df_F splits and n_splits
        df_P, df_F, n_split = make_seperate_n_splits(daf)

        # If n_split == 1, add 1.
        if n_split == 1:
            n_split+=1 


        df_list = make_balanced_dataset(n_split, df_F)


        n_iter = 0
        # as many for statements as there are datasets created. 
        LR_random_list = []
        xgb_random_list=[]
        X_valid_list=[]
        y_valid_list=[]
        for i in range(n_split):
            # iteration as many times as there are datasets
            n_iter += 1
            y_label = df_list[i]['post_GN']
            drop_data = df_list[i].drop(columns=['post_GN'])            

            X_data = np.array(drop_data.iloc[:, :])
            
            y_label_len = len(y_label)
            make_logistic(X_data, y_label, 5, i, q)            


        # select & predict with models with large (TP+TN)-(FP+FN) values
        for j in range(len(LR_random_list)): 

            y_pred = LR_random_list[j].predict(X_valid_list[j])

            cm = confusion_matrix(y_valid_list[j], y_pred, labels=[1, 0]).ravel()
            TP_value, FN_value, FP_value, TN_value = cm.ravel()

            valid_roc_auc_scores = calc_valid_auc(TP_value, FP_value, FN_value, TN_value)
            valid_roc_auc_scores_list.append(valid_roc_auc_scores)

            max_count = (TP_value + TN_value) - (FN_value + FP_value)
            max_count_list.append(max_count)        
        

        # Calculate confidence intervals
        base_score = max(max_count_list) * 0.999
        base_score_list.append(base_score)

        # Calculate AUC
        base_auc_score = max(valid_roc_auc_scores_list) * 0.999
        base_auc_score_list.append(base_auc_score)

        # of models above max auc score (number to ensemble)
        score_auc_count = np.where(valid_roc_auc_scores_list > base_auc_score)[0]
        score_auc_count_list.append(len(score_auc_count))
        
        o = max_count_list.count(max(max_count_list))
        max_count_score_list.append(o)

        # extract model index higher than base_score
        model_index = make_max_score_model_selection(max_count_list, base_score)

        # Ensemble | prediction proba average (X_test) / return y_pred 
        mean_y_pred_test = cal_pred_proba_test(model_index)


        # Evaluation metrics (test)
        macro_score = f1_score(mean_y_pred_test, y_test, average='macro')

        cm_test = confusion_matrix(y_test, mean_y_pred_test, labels=[1, 0])
        cm_test_list.append(cm_test)

        
    # include only exact matches for the entire list.
    matches = sum(i == j for i, j in zip(original_test_inv_data_list, label_y_test_list)) # 일치한 것 확인
    total = len(original_test_inv_data_list)

    match_rate = matches / total

    print(f'All Match rate: {match_rate*100:.2f}%')
    
    matches_w_pre_inv = sum(i == j for i, j in zip(original_test_inv_data_list, X_test_data_pre_invasion)) # 일치한 것 확인
    total = len(original_test_inv_data_list)

    match_rate_w_pre_inv = matches_w_pre_inv / total

    print(f'imputated vs pre invasion, Match rate: {match_rate_w_pre_inv*100:.2f}%')


    # store the average of the base scores from the valid set from each fold
    base_score_rs_list.append(np.mean(base_score_list))

    # Calculate TP, FP, FN, TN, roc_auc
    sum_TP, sum_FP, sum_FN, sum_TN, roc_auc_scores_test = make_cm_list_5(cm_test_list)
    roc_auc_scores_test_list.append(roc_auc_scores_test)
    print()



roc_auc_scores_test_list_list.append(roc_auc_scores_test_list)
base_score_list_list.append(base_score_list)
max_count_score_list_list.append(max_count_score_list)


False    184
True      67
Name: post_GN, dtype: int64
************ Group 1 ***************
Index(['Age', 'CA125 (IU/ml)', 'Tumor size (largest diameter)', 'Grade',
       'Myometrial invasion depth', 'Menopause_No', 'Menopause_Yes',
       'Method_Dilatation/currettage', 'Method_Hysteroscopy',
       'Method_Pipelle biopsy', 'pre_Myometrial invasion depth'],
      dtype='object')
Index(['Age', 'CA125 (IU/ml)', 'Tumor size (largest diameter)', 'Grade',
       'Myometrial invasion depth', 'Menopause_No', 'Menopause_Yes',
       'Method_Dilatation/currettage', 'Method_Hysteroscopy',
       'Method_Pipelle biopsy'],
      dtype='object')
All Match rate: 58.17%
imputated vs pre invasion, Match rate: 100.00%
TP: 48,  FP: 68, FN: 19, TN: 116
0.6734

False    237
True      14
Name: post_GN, dtype: int64
************ Group 2 ***************
Index(['Age', 'CA125 (IU/ml)', 'Tumor size (largest diameter)', 'Grade',
       'Myometrial invasion depth', 'Menopause_No', 'Menopause_Yes',
       'Method

# Calculate confidence intervals using bootstrapped

In [24]:
import sys
import sklearn

print("Python version:", sys.version)
print("scikit-learn version:", sklearn.__version__)


Python version: 3.7.13 (default, Mar 28 2022, 08:03:21) [MSC v.1916 64 bit (AMD64)]
scikit-learn version: 1.0.2


In [25]:
import numpy as np
from sklearn.metrics import roc_auc_score, cohen_kappa_score
from sklearn.utils import resample

# Define the confusion matrix values for each group
# TP, FP, FN, TN
confusion_matrices = [
    (48, 68, 19, 116),  # Group1
    (11, 114, 3, 123),  # Group2
    (77, 102, 21, 51),  # Group3
    (24, 79, 13, 135)   # Group4
]

# Number of bootstrap iterations and confidence level
n_iterations = 10000
alpha = 0.05

# Function to calculate bootstrap confidence intervals
def bootstrap_confidence_intervals(matrix):
    TP, FP, FN, TN = matrix

    # Actual labels and predicted labels
    y_true = np.concatenate([np.ones(TP + FN), np.zeros(FP + TN)])
    y_pred = np.concatenate([np.ones(TP), np.zeros(FN), np.ones(FP), np.zeros(TN)])

    aucs = []
    

    for _ in range(n_iterations):
        # Resample
        y_true_sample, y_pred_sample = resample(y_true, y_pred)

        # Calculate AUC
        auc = roc_auc_score(y_true_sample, y_pred_sample)
        aucs.append(auc)


    # Sort and calculate confidence intervals
    aucs.sort()

    lower_idx = int(n_iterations * alpha / 2)
    upper_idx = int(n_iterations * (1 - alpha / 2))

    # Calculate mean and confidence intervals
    results = {
        'AUC Mean': np.mean(aucs),
        'AUC CI': (aucs[lower_idx], aucs[upper_idx]),
    }
    return results

# Calculate and store results for each group
group_results = {}
for i, matrix in enumerate(confusion_matrices):
    group_results[f'Group{i + 1}'] = bootstrap_confidence_intervals(matrix)

# Print results for each group with formatting
for group, results in group_results.items():
    print(f"{group}:")
    print(f"  AUC Mean: {results['AUC Mean']:.4f}, 95% CI: ({results['AUC CI'][0]:.4f}, {results['AUC CI'][1]:.4f})")
    print()


Group1:
  AUC Mean: 0.6742, 95% CI: (0.6083, 0.7367)

Group2:
  AUC Mean: 0.6516, 95% CI: (0.5255, 0.7605)

Group3:
  AUC Mean: 0.5594, 95% CI: (0.5034, 0.6149)

Group4:
  AUC Mean: 0.6405, 95% CI: (0.5557, 0.7210)

